In [146]:
# import dependencies
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import requests
from icecream import ic

import os
from dotenv import load_dotenv

from ta import add_all_ta_features
from ta.trend import MACD
from ta.volatility import BollingerBands
from ta.volume import VolumeWeightedAveragePrice
from ta.momentum import StochRSIIndicator

import backtrader as bt

import IPython
import IPython.display

pd.plotting.register_matplotlib_converters()


## Get Data

In [147]:
def get_historical_price(tickers, data_type):

    # pulls historical daily or intraday OLHC prices and volume

    d = {}

    for i in range(len(tickers)): 

        r = requests.get('https://eodhistoricaldata.com/api' + '/' + data_type + '/' + tickers[i] + '.US', 
            params={'api_token': '63dc0e2f4efc43.34327983', 'fmt': 'json'}
            )
        data = r.json()
        r.close()

        # ADD PROGRESS BAR
        
        for symbol in tickers:
            d[symbol] = pd.DataFrame.from_records(data)

            indicator_macd = MACD(close = d[symbol]['close'], window_slow=26, window_fast=12, window_sign=9, fillna=True)
            d[symbol]['macd'] = indicator_macd.macd()
            d[symbol]['macd_diff'] = indicator_macd.macd_diff()
            d[symbol]['macd_signal'] = indicator_macd.macd_signal()

            indicator_bb = BollingerBands(close = d[symbol]['close'], window=20, window_dev=2, fillna=True)
            d[symbol]['bb_mavg'] = indicator_bb.bollinger_mavg()
            d[symbol]['bb_hband'] = indicator_bb.bollinger_hband()
            d[symbol]['bb_lband'] = indicator_bb.bollinger_lband()
            d[symbol]['bb_hband_ind'] = indicator_bb.bollinger_hband_indicator()
            d[symbol]['bb_lband_ind'] = indicator_bb.bollinger_lband_indicator()

            indicator_vwap = VolumeWeightedAveragePrice(
                high = d[symbol]['high'],
                low = d[symbol]['low'],
                close = d[symbol]['close'],
                volume = d[symbol]['volume'],
                window=14, fillna=True)
            d[symbol]['vwap'] = indicator_vwap.volume_weighted_average_price()

            indicator_stochrsi = StochRSIIndicator(close = d[symbol]['close'], window=14, smooth1=3, smooth2=3, fillna=True)
            d[symbol]['stoch_rsi'] = indicator_stochrsi.stochrsi()
            d[symbol]['stochrsi_d'] = indicator_stochrsi.stochrsi_d()
            d[symbol]['stochrsi_k'] = indicator_stochrsi.stochrsi_k()

    return d

In [148]:
# placeholder tickers
tickers = ['SPXL', 'SPXS']

# get intraday prices | dictionary of dataframes
intraday_data = get_historical_price(tickers, 'intraday')

# get daily prices | dictionary of dataframes
daily_data = get_historical_price(tickers, 'eod')

In [149]:
# concat intraday dataframes together and convert to dask df
intraday_df = pd.concat(intraday_data.values(), axis=1, keys=intraday_data.keys())
intraday_dask_df = dd.from_pandas(intraday_df, npartitions=6)

# concat daily dataframes together and convert to dask df
daily_df = pd.concat(daily_data.values(), axis=1, keys=daily_data.keys())
daily_dask_df = dd.from_pandas(daily_df, npartitions=6)

## Backtrader

In [150]:
df = daily_df['SPXL'].set_index('date')

df = df.copy()

In [151]:
df = df[:200].copy()
#df.drop(['timestamp', 'gmtoffset'], axis = 1)


In [152]:
def plot_system1(data):

    # come back and write loop for spxs
    df = data.copy()
    dates = df.index
    #price = df['Adj Close']
    #sma200 = df['sma200']
    #ema20 = df['ema20']
    
    with plt.style.context('fivethirtyeight'):
        fig = plt.figure(figsize=(14,7))
        plt.plot(dates, df.close, linewidth=1.5, label='Close Price')
        plt.plot(dates, df.bb_hband, linewidth=2, label='BB High')
        plt.plot(dates, df.bb_mavg, linewidth=2, label='MA')
        plt.plot(dates, df.bb_lband, linewidth=2, label='BB Low')
        plt.title("Mean Reversion")
        plt.ylabel('Price($)')
        plt.legend()
    
    plt.show() # This is needed only if not in Jupyter

In [153]:
plot_system1(df)

<IPython.core.display.Javascript object>

In [154]:
def plot_system1_sig(data):
    df = data.copy()
    dates = df.index
    # price = df['Adj Close']
    # sma200 = df['sma200']
    # ema20 = df['ema20']
    
    buy_signals = (df.bb_lband_ind == 1) & (df.bb_lband_ind.shift(1) == 0)
    buy_marker = df.bb_mavg * buy_signals - (df.bb_mavg.max()*.05)
    buy_marker = buy_marker[buy_signals]
    buy_dates = df.index[buy_signals]
    sell_signals = (df.bb_hband_ind == 1) & (df.bb_hband_ind.shift(1) == 0)
    sell_marker = df.bb_mavg * sell_signals + (df.bb_mavg.max()*.05)
    sell_marker = sell_marker[sell_signals]
    sell_dates = df.index[sell_signals]
    
    with plt.style.context('fivethirtyeight'):
        fig = plt.figure(figsize=(14,7))
        plt.plot(dates, df.close, linewidth=1.5, label='Close Price')
        plt.plot(dates, df.bb_hband, linewidth=2, label='BB High')
        plt.plot(dates, df.bb_mavg, linewidth=2, label='MA')
        plt.plot(dates, df.bb_lband, linewidth=2, label='BB Low')
        plt.scatter(buy_dates, buy_marker, marker='^', color='green', s=160, label='Buy')
        plt.scatter(sell_dates, sell_marker, marker='v', color='red', s=160, label='Sell')
        plt.title("A Simple Crossover System with Signals")
        plt.ylabel('Price($)')
        plt.legend()
    
    plt.show() # This is needed only if not in Jupyter

In [155]:
plot_system1_sig(df)

<IPython.core.display.Javascript object>

In [156]:
# The returns of the Buy and Hold strategy:
df['Hold'] = returns = (df.close - df.close.shift(1))/df.close.shift(1)
# The returns of the Moving Average strategy:
df['Strategy'] = df.bb_lband_ind.shift(1) * df['Hold']
# We need to get rid of the NaN generated in the first row:
df.dropna(inplace=True)
df

,open,high,low,close,adjusted_close,volume,macd,macd_diff,macd_signal,bb_mavg,bb_hband,bb_lband,bb_hband_ind,bb_lband_ind,vwap,stoch_rsi,stochrsi_d,stochrsi_k,Hold,Strategy
date,,,,,,,,,,,,,,,,,,,,
2008-11-06,75.00,80.5000,69.85,79.9400,97918.1755,99,0.832023,0.665618,0.166405,74.725000,85.155000,64.295000,0.0,0.0,75.777320,0.000000,0.000000,0.000000,0.150050,0.0
2008-11-07,78.39,79.8300,73.25,73.9499,90580.9267,267,0.996568,0.664131,0.332437,74.466633,83.013989,65.919277,0.0,0.0,75.706011,0.000000,0.000000,0.000000,-0.074932,-0.0
2008-11-10,69.64,79.3200,67.89,75.5800,92577.6417,194,1.244165,0.729382,0.514783,74.744975,82.209736,67.280214,0.0,0.0,75.215855,0.000000,0.000000,0.000000,0.022043,0.0
2008-11-11,80.27,85.4000,76.24,80.2500,98297.8932,281,1.796508,1.025380,0.771128,75.845980,83.844325,67.847635,0.0,0.0,77.001506,0.000000,0.000000,0.000000,0.061789,0.0
2008-11-12,87.09,94.1024,84.30,93.0000,113915.3155,283,3.225876,1.963799,1.262078,78.704983,93.428749,63.981218,0.0,0.0,80.359098,0.000000,0.000000,0.000000,0.158879,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-08-17,27.31,27.8500,27.22,27.7100,33941.8644,8033,-2.013373,0.307527,-2.320900,27.287100,30.460074,24.114126,0.0,0.0,26.616537,1.000000,0.773331,0.837374,0.074447,0.0
2009-08-18,27.53,27.6890,26.75,27.0000,33072.1884,7570,-1.845915,0.379988,-2.225903,27.097100,29.830552,24.363648,0.0,0.0,26.462363,0.866298,0.841137,0.948297,-0.025623,-0.0
2009-08-19,27.72,27.7499,26.07,26.2500,32153.5165,9864,-1.753509,0.377915,-2.131425,26.873600,29.062295,24.684905,0.0,0.0,26.383477,0.732821,0.884014,0.866373,-0.027778,-0.0


## New Backtrader

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
if __name__ == '__main__':

    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname='data/TSLA.csv',
        # Do not pass values before this date
        fromdate=datetime.datetime(2015, 6, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2019, 6, 2),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(1000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
    cerebro.plot()

Starting Portfolio Value: 1000.00
2015-07-17, Close, 18.31
2015-07-17, BUY CREATE, 18.31
2015-07-20, BUY EXECUTED, Price: 18.33, Cost: 183.30, Comm 0.18
2015-07-20, Close, 18.82
2015-07-21, Close, 17.78
2015-07-21, SELL CREATE, 17.78
2015-07-22, SELL EXECUTED, Price: 17.42, Cost: 183.30, Comm 0.17
2015-07-22, OPERATION PROFIT, GROSS -9.10, NET -9.46
2015-07-22, Close, 17.86
2015-07-22, BUY CREATE, 17.86
2015-07-23, BUY EXECUTED, Price: 17.98, Cost: 179.80, Comm 0.18
2015-07-23, Close, 17.81
2015-07-23, SELL CREATE, 17.81
2015-07-24, SELL EXECUTED, Price: 17.83, Cost: 179.80, Comm 0.18
2015-07-24, OPERATION PROFIT, GROSS -1.50, NET -1.86
2015-07-24, Close, 17.69
2015-07-27, Close, 16.87
2015-07-28, Close, 17.65
2015-07-29, Close, 17.59
2015-07-30, Close, 17.79
2015-07-30, BUY CREATE, 17.79
2015-07-31, BUY EXECUTED, Price: 17.84, Cost: 178.40, Comm 0.18
2015-07-31, Close, 17.74
2015-07-31, SELL CREATE, 17.74
2015-08-03, SELL EXECUTED, Price: 17.75, Cost: 178.40, Comm 0.18
2015-08-03, OPE

<IPython.core.display.Javascript object>

In [ ]:
## Cant get it to plot 

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import argparse

import backtrader as bt
import backtrader.feeds as btfeeds

import pandas


def runstrat():
    args = parse_args()

    # Create a cerebro entity
    cerebro = bt.Cerebro(stdstats=False)

    # Add a strategy
    cerebro.addstrategy(bt.Strategy)

    # Get a pandas dataframe
    datapath = (df)

    # Simulate the header row isn't there if noheaders requested
    skiprows = 1 if args.noheaders else 0
    header = None if args.noheaders else 0

    dataframe = pandas.read_csv(datapath,
                                skiprows=skiprows,
                                header=header,
                                parse_dates=True,
                                index_col=0)

    if not args.noprint:
        print('--------------------------------------------------')
        print(dataframe)
        print('--------------------------------------------------')

    # Pass it to the backtrader datafeed and add it to the cerebro
    data = bt.feeds.PandasData(dataname=dataframe)

    cerebro.adddata(data)

    # Run over everything
    cerebro.run()

    # Plot the result
    cerebro.plot(style='bar')


def parse_args():
    parser = argparse.ArgumentParser(
        description='Pandas test script')

    parser.add_argument('--noheaders', action='store_true', default=False,
                        required=False,
                        help='Do not use header rows')

    parser.add_argument('--noprint', action='store_true', default=False,
                        help='Print the dataframe')

    return parser.parse_args()


if __name__ == '__main__':
    runstrat()

usage: ipykernel_launcher.py [-h] [--noheaders] [--noprint]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"59680222-86ac-4a65-a42a-5b322e43f364" --shell=9002 --transport="tcp" --iopub=9004 --f=/home/groovyjac/.local/share/jupyter/runtime/kernel-v2-170SobPVuJhrC24.json


SystemExit: 2

/home/groovyjac/miniconda3/envs/honors-thesis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Broke: this just adds optimization

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])


# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

In [ ]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.optstrategy(
        TestStrategy,
        maperiod=range(10, 31))

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname='data/TSLA.csv',
        # Do not pass values before this date
        fromdate=datetime.datetime(2015, 5, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2020, 1, 31),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(1000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Run over everything
    cerebro.run(maxcpus=1)

AttributeError: module 'collections' has no attribute 'Iterable'